# 4.2 Blob detection and description

Now, we are going to deal with another type of keypoints. Corners and edges were already covered, so it is time for **blobs**!  

Informally, a blob is a region of an image in which some properties are constant or approximately constant. All the points in a blob can be considered in some sense to be similar to each other. Blobs provide complementary information about regions, which is not obtained from edges or corners.

The **S**cale-**I**nvariant **F**eature **T**ransform (**SIFT**) is a blob detection and description algorithm whose implementation is far more complex than the Harris detector but easier to use in openCV. In this notebook we will understand the main steps involved in the SIFT algorithm and how can it be implemented.

## Problem context - Image stitching

In notebook 4.1, the Harris corner detector + NCC was proposed to find correspondences between images for an image stitching system. But this is just not enough, as matching keypoints through NCC is not **invariant** against typical transformations between images such as **rotation**, **scale** or change in the **point of view**. In these cases, it is not possible to find matches between the transformed images. Besides, it does not differentiate points in repetitive elements (e.g. windows in a skyscraper):$\\[10pt]$

<img src="./images/ncc_problem.jpg" width="300" align="left"/><img src="./images/skyscraper.jpg" width="300" align="rigth"/>

## SIFT (Scale-Invariant Feature Transform)

As SIFT is patented, it is not included in the OpenCV. So, we are going to use **[pysift](https://github.com/rmislam/PythonSIFT)**, a third party software that implements it like an OpenCV method (only for educational purposes), returning our beloved [cv2.KeyPoint](https://docs.opencv.org/3.4/d2/d29/classcv_1_1KeyPoint.html) objects.

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
images_path = './images/'

# import sys
# sys.path.append("..")
# from utils.third_party import pysift #https://github.com/rmislam/PythonSIFT

The first to understand is **how the SIFT detector works**?

### Step 1: Contruction of octaves

SIFT progressivelly blurs the input image by applying a set of Gaussian filters with a growing standard deviation. Once you have a sequence of $s$ images (i.e. a sequence of 6 blurred images in the example below), we have completed an **octave**. Then, the image resolution is reduced to half and the blurring process continues. We repeat this process until the pictures are too small to proceed$\\[5pt]$ (i.e. we pre-define a certain number of octaves):

<img src="./images/sift_octave.png" width="800"/>$\\[5pt]$

The blue arrow points to the beginning of a new octave while the green arrow indicates the process of Gaussian blurring. As you can see, in the example we have octaves of 6 images ($s = 6$) and 4 octaves (i.e. the resolution is downsampled 4 times). If we stack all the images, we say that we have built a pyramid of images:

<img src="./images/sift_dog.jpg" width="400"/>$\\[5pt]$

### Step 2: Difference of Gaussians (DoG)

For each pair of horizontally adjacent pictures in the table above, we compute the differences of the individual pixels. In this way, we have of $(s-1)$ DoG images for each octave:$\\[5pt]$

<img src="./images/octave_dog.png" width="800"/>$\\[5pt]$

The **Difference of Gaussians** (DoG) method is a different and more efficient way to implement the LoG (laplacian of Gaussian) border detector, which was covered in notebook 3.2.

### Step 3: Extreme points

Now, for detecting blobs in SIFT, we search for **extreme** points in the DoG images. For that, each pixel is compared with its **26 neighbors** (8 in the same image, 9 in the previous image in the pyramid and 9 in the next one). An extreme point can be a maximum or a minimum, so, if the grey level of the pixel is greater or lower than all its 26 neighbors it is considered as a keypoint candidate.

*Note that this is only possible for DoG images that are not the first or the last image in that scale.*$\\[5pt]$

<img src="./images/sift_keypoints.png" width="500"/>$\\[5pt]$

Local maximum points have been plot in yellow while local minimum points are in red. An example of an extreme point can be seen next:

<img src="./images/neighbors.png" width="500"/>$\\[5pt]$

### Step 4: Compute descriptors

Finally, the SIFT keypoints descriptor are computed as the histogram of orientations around the extreme point. So, first the orientation at each pixel in the local neighborhood is computed and inserted into an histogram, but having weighted it by two factors:

- The magnitude of the gradient
- A Gaussian centered at the extreme point

This way we give more importance to the orientations of those pixels closer to the candidate and those pixels with a higher score (i.e. they have *stronger* orientations).

<img src="./images/sift_descriptor.png" width="800"/>$\\[5pt]$

The histogram has a pre-defined number of bins so that the values of such bins of orientations will be considered as the **descriptor** of the keypoints. The mode of the histogram is also stored as the **main orientation** of the keypoint. All this information, along with the **scale** in which the extrema was found, will be useful to perform keypoint matches.

Of course, this has been a short resume where a lot of small details have been skipped, but we think you'll get the gist of it. If you want a fully detailed explanation of the algorithm, have a look to [this](http://weitz.de/sift/index.html?size=large).


## Coding

But coming back to coding, ~~`pysift` defines a main function `pysift.computeKeypointsAndDescriptors` whose only input is a grey image~~, we will use the implementation of SIFT within the OpenCV. For that, create a SIFT detector with `sift = cv2.xfeatures2d.SIFT_create()` and then call to the method `detectAndCompute(image,mask)`, as explained [here](https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html). This function already returns a list of `cv2.KeyPoint` objects and one array with shape $\left(n\_keypoints,4x4x8\right)$ that corresponds to the 128D descriptor of each keypoint.

**<span style="color:red">EXERCISE 4.2.1 -- </span>** Find SIFT keypoints in the `sift_2.jpeg` image and display the image along with the keypoints.

*Note that for drawing complex keypoints (as those obtained using SIFT), we can use `flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS` in the drawing method*. *Tip: finding SIFT keypoints is a really costly task, so it will take some time to finish.*

In [1]:
# EXERCISE 4.2.1
# Find SIFT keypoints in 'sift_2.jpeg' image and display them using 'flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS'
# Use the grey image for finding the keypoints but display its color version.
# Write your code here!

Now, we are going to match the keypoints in two images of a mountain: `sift_1.jpeg` and `sift_2.jpeg`, which have overlapping fields.$\\[5pt]$

<img src="./images/mountain_1.jpg" width="300" align="left"/><img src="./images/mountain_2.jpg" width="300" align="rigth"/>$\\[5pt]$

*For this exercise, the images have been downsampled in order to compute keypoints faster*

**What to do?** Follow the next steps to develop a keypoint matching system using SIFT descriptors.

Similar to harris, we are going to do this step by step:

**<span style="color:red">EXERCISE 4.2.2a -- </span>**
- **First compute keypoints and descriptors for each image.**

In [2]:
# EXERCISE 4.2.2a
# Load the images and compute SIFT keypoints
# Write your code here!

**<span style="color:red">EXERCISE 4.2.2b -- </span>**
- **Now, find matches between keypoints**

*When you match keypoints, you actually match their descriptors.*

Remember when we used the Brute-Force matcher for the ORB keypoints? Now we are going one step further and use a `knnMatch` ($K$-Nearest Neighbors). This will also return a list of the best $K$ matches but looking for them more *efficiently*. If we use $K=2$ we will get 2 matches for each keypoint, the 2 nearests matches. The distance between two descriptors is given at `DMatch.distance`. If the distance is very low, then the keypoints are similar!

Use `cv2.BFMatcher().knnMatch()` [(documentation)](https://docs.opencv.org/3.4/db/d39/classcv_1_1DescriptorMatcher.html#a378f35c9b1a5dfa4022839a45cdf0e89) to obtain the list of matches, then print the first match and their distances.

In [3]:
# EXERCISE 4.2.2b
# Use 'knnMatch' to find matches and print the distances between the first keypoint in the first image and the two most similar in the second one.
# Write your code here!

**<span style="color:red">EXERCISE 4.2.2c -- </span>**
- **Filter non-robust matches**

In order to discard incorrect matches, we are going to use the distance value for the two output matches that we have found for each keypoints. Thus, we will only accept matches that has not two close solutions (i.e. with similar distance), because that would mean that, for a certain keypoint in one image, there are two different keypoints that are very similar to it (in terms of descriptor distance). In this situation, it is advisable to discard the match, as a wrong match (usually called **outlier**) can severely affect the robustness of any feature-based computer vision system.

So, iterate along the matches list and filter those matches that does not fulfil the specified condition (define a certain threshold to discard wrong matches).

In [4]:
# EXERCISE 4.2.2c
# Iterate over the matches and select those that seems to be robust matches.
# Write your code here!

**<span style="color:red">EXERCISE 4.2.2d -- </span>**
- Finally, **display the images drawing before computed (and filtered) matches**

*Remember to use `flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS`*

In [5]:
# EXERCISE 4.2.2d
# Display the image and the matches with 'flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS'
# Write your code here!

**Answer following questions**

- Looking at the results, could a system stitch both images?  
  
- How do you think it could be done?

**<span style="color:blue">(Answer these questions here!)</span>**

## Conclusion

SIFT is patented and you are suposed to pay it for its use. There are some official (and also more efficient) alternatives in OpenCV like [SURF](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_surf_intro/py_surf_intro.html). Anyway, SIFT is one of the **best algorithms for feature description** and its robust against changes in light conditions, scale, rotation and partially to changes in viewpoint, it is mandatory to learn how it was acomplished and how it works.


### Extra

If you want to learn more about SIFT, you can look through the [original paper](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=2ahUKEwi7y_G-xrDoAhUGHcAKHWn1DIsQFjAAegQIBRAB&url=https%3A%2F%2Fwww.cs.ubc.ca%2F~lowe%2Fpapers%2Fijcv04.pdf&usg=AOvVaw0cBGKtFZfTh8Vf_Y4kP3LS) by David Lowe. It was proposed in 1999 and published in 2004.